# Atlas Search (for SI Genie)

#### dependency

In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel

#### string constant, variable defenition

In [2]:
MONGODB_DATABASE_NAME = "si_genie"
MONGODB_COLLECTION_NAME = "bkg"

SEARCH_INDEX = 'bkg_search_index'

### Mongo DB

In [3]:
load_dotenv()
MONGODB_CONNECTION_STRING = os.environ["MONGODB_URI"]

mongo_client = MongoClient(MONGODB_CONNECTION_STRING)
mongo_db = mongo_client[MONGODB_DATABASE_NAME]
bkg_collection = mongo_db[MONGODB_COLLECTION_NAME]

##### 데이터 예시
```python
{
    "_id": {"$oid":"66e184160d94dd72a1a86d47"},
    "bookingReference": "CHERRY202409128925",
    "customerName": "SUNRISE IMPORTS LTD",
    "shipperName": "SUNRISE IMPORTS LTD",
    "invoiceReceiver": "SUNRISE TRADING INC.",
    "voyageDetails": {
        "vesselName": "CMA CGM GEORG FORSTER",
        "voyageNumber": "2409EWS"
    },
    "cargoDetails": {
        "hsCode": "847989",
        "chapterDescription": "Machinery for working metal; tools for working in the hand, ...",
        "commodity": "METALWORKING TOOLS"
    },
    "containerDetails": {
        "size": "40 HIGH CONTAINER",
        "type": "DRY",
        "quantity": 2
    },
    "routeDetails": {
        "placeOfReceipt": "NINGBO, CHINA",
        "portOfLoading": "NINGBO, CHINA",
        "portOfDischarge": "ROTTERDAM, NETHERLANDS",
        "placeOfDelivery": "ROTTERDAM, NETHERLANDS"
    },
    "scheduleDetails": {
        "estimatedArrivalAtLoadingPort": "2024-10-18 14:00",
        "estimatedDepartureFromLoadingPort": "2024-10-19 08:00",
        "estimatedArrivalAtDischargePort": "2024-11-07 12:00"
    },
    "emptyContainerPickupLocation": "NINGBO, CHINA",
    "transportationTerm": "CIF",
    "remarks": "D/G, Class: 9, UN No: 3334",
    "filename": "bkg_CHERRY202409128925.json"
}
```

### Create Index

In [42]:
# List up the search indices for the collection
bkg_search_indices = bkg_collection.list_search_indexes().to_list()
bkg_search_indices_names = [dic['name'] for dic in bkg_search_indices]
print("\n<Search Index>")
for dic in bkg_search_indices:
    print(f"{dic['name']} ::: 'definition': {dic['latestDefinition']}")

# Create dynamic search index if it doesn't exist
if SEARCH_INDEX not in bkg_search_indices_names:
    print(f"Index '{SEARCH_INDEX}' not found. Creating index ...")

    # list of stopwords
    stopwords_list = [ "booking", "cargo", "commodity", "container", "customer", "document", "place", "remark", "shipper", "vessel", "voyage" ]

    # Search Index 정의
    search_index_model = SearchIndexModel(
        definition={
            'analyzer': "string_analyzer",          # 인덱싱에 사용할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
            'searchAnalyzer': "string_analyzer",     # 검색 input을 분석할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
            'mappings': {                        # 인덱싱 대상 필드 지정
                'dynamic': True,                 # 동적 맵핑 - 모든 필드를 대상으로 지정
                'fields': {                      # 정적 맵핑할 필드 지정
                    'bookingReference': [                 # 'bookingReference' 필드
                        {
                            'type': "string",                       # 데이터 타입 - 문자열 타입
                            'analyzer': "serial_number_analyzer",   # 인덱싱에 사용할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
                            'searchAnalyzer': "serial_number_analyzer"      # 검색 input을 분석할 analyzer 지정 : Atlas Search 내장 analyzer or custom analyzer
                        }
                    ],
                    'voyageDetails.voyageNumber': [       # 'voyageDetails' 객체 하위 'voyageNumber' 필드
                        {
                            'type': "string",
                            'analyzer': "serial_number_analyzer",
                            'searchAnalyzer': "serial_number_analyzer"
                        }
                    ],
                    'cargoDetails.hsCode': [              # 'cargoDetails' 객체 하위 'hsCode' 필드
                        {
                            'type': "string",
                            'analyzer': "serial_number_analyzer",
                            'searchAnalyzer': "serial_number_analyzer"
                        }
                    ]
                }
            },
            'analyzers': [                          # 이 인덱스에서 사용할 custom analyzer 정의 - 여러 개 선언 가능
                {
                    'name': "string_analyzer",      # 문자열 데이터용 analyzer
                    'charFilters': [                # 데이터 검사 및 필터링 작업 설정
                        {
                            'type': "mapping",      # 지정된 문자를 다른 문자로 대체하는 방식
                            'mappings': {           # 'mapping' 타입 추가 옵션
                                ",": ""             # comma ","를 공백문자 ""로 대체
                            }
                        }
                    ],
                    'tokenizer': {                  # 텍스트 데이터를 개별 청크로 분할하는 방식 설정
                        'type': "regexSplit",       # tokenizer 타입 - 정규표현식 기반의 seperator를 사용하여 청크 분할
                        'pattern': "[-_. ]+"        # 'regexSplit' 타입 추가 옵션 : 정규표현식 패턴 지정
                    },
                    'tokenFilters': [                       # 토큰 분할 후 필터링 작업 설정
                        {
                            'type': "lowercase"             # 대문자 -> 소문자 변환
                        },
                        {
                            'type': "englishPossessive"     # 단어 끝의 소유격('s) 제거
                        },
                        {
                            'type': "kStemming"             # 단어의 실질적인 의미를 가지는 어간만 필터링 (ex. meetings -> meeting)
                        },
                        {
                            'type': "length",               # 토큰 길이 제한 : 최소/최대 길이 지정
                            'min': 3                        # 'length' 타입 추가 옵션 : 최소 길이 제한
                        },
                        {
                            'type': "stopword",             # 지정된 단어에 해당하는 토큰 제거
                            'tokens': stopwords_list,       # 'stopword' 타입 추가 옵션 : 제거할 단어 목록 지정
                            'ignoreCase': True              # 'stopword' 타입 추가 옵션 : 대소문자 구분 무시 여부 - True로 지정하면 "cargo" == "Cargo"로 인식함
                        },
                        {
                            'type': "porterStemming"        # 토큰에서 일반적인 형태소 및 굴절 접미사 제거 ex. meeting -> meet / weekly -> weekli
                        },
                        {
                            'type': "edgeGram",             # 하위 옵션으로 지정된 길이만큼 토큰의 왼쪽 가장자리부터 토큰화
                            'minGram': 3,                   # ex. regular -> reg, regu, regul, regula, regular
                            'maxGram': 7                    # technique -> tec, tech, techn, techni, techniq
                        }
                    ]
                },
                {
                    'name': "serial_number_analyzer",       # 일련번호 데이터용 analyzer
                    'charFilters': [
                        {
                            'type': "mapping",
                            'mappings': {
                                "_": "",
                                "-": "",
                                ".": "",
                                " ": ""
                            }
                        }
                    ],
                    'tokenizer': {
                        'type': "standard"
                    },
                    'tokenFilters': [
                        {
                            'type': "lowercase"
                        }
                    ]
                }
            ]
        },
        name=SEARCH_INDEX,      # Search Index명 지정
        type="search"           # Index 유형 지정 : "search" -> search index / "vectorSearch" -> vector search index
    )

    # Search Index 생성
    bkg_collection.create_search_index(model=search_index_model)
    
    print("Index creation finished.")


<Search Index>
Index 'bkg_search_index' not found. Creating index ...
Index creation finished.


### run search

In [53]:
# QUERY = ""
# QUERY = "GLOBAL TRADING SOLUTIONS"
# QUERY = "reefer container"
# QUERY = "reefer"
# QUERY = "container"
# QUERY = "dry container"
# QUERY = "dry"
QUERY = "high cubic"
# QUERY = "flat"
# QUERY = "2409EWS"
# QUERY = "243W"

# Atlas Search 집계 파이프라인
search_pipeline = [
    {
        '$search': {                        # 검색 단계
            'index': SEARCH_INDEX,          # 검색에 사용할 Search Index 명
            'compound': {                   # 복합 검색
                'should': [                 # 복합 검색 연산자 - 'should' : OR 연산 -> 각 하위 쿼리 결과의 점수 합산
                    {
                        'text': {                       # 검색 유형 지정 - 'text'==본문 검색
                            'query': QUERY,             # 검색 input
                            'path': {'wildcard': "*"},  # 검색 대상 필드 경로 - wildcard를 이용하여 모든 필드를 대상으로 지정
                            'matchCriteria': "all",     # 입력값과의 일치 기준 - 'all' : 입력값의 모든 token을 포함하는 문서만 검색되도록 설정
                        }
                    },
                    {
                        'text': {
                            'query': QUERY,
                            'path': ["bookingReference", "voyageDetails.voyageNumber", "cargoDetails.hsCode"], # 검색 대상 필드 경로
                            'matchCriteria': "all",
                            'score': { 'boost': { 'value': 10 }}    # 검색 조건과 일치하면 가산점
                        }
                    }
                ]
            },
            'concurrent': True,         # 검색 병렬 실행 요청 여부
            'highlight': {                  # 검색 결과 강조 표시 설정
                'path': {'wildcard': "*"}   # 강조 표시 대상 필드 경로 - wildcard를 이용하여 모든 필드를 대상으로 지정
            }
        }
    },
    {
        '$set': {                                           # 결과 데이터 수정 단계 - metadata를 함께 출력하기 위함
            'score': { '$meta': "searchScore" },            # 검색 결과 점수 필드 추가
            'highlights': { '$meta': "searchHighlights" }   # 검색 결과 강조 표시 데이터 추가 : 검색어와 일치한 부분에 대한 정보
        }
    },
    {
        '$project': {       # projection 단계 : 검색 결과로 출력할 필드 지정
            '_id': 0        # '_id' 필드 제외, 그외 모든 필드 포함
        }
    }
]

results = bkg_collection.aggregate(pipeline=search_pipeline).to_list()

# for d in results[-100:]:
for d in results:
    # print(d)
    print(f"{d['bookingReference']}")
    # print(f"customer ::: {d['customerName']}")
    # print(f"vessel name ::: {d['voyageDetails']['vesselName']}")
    # print(f"voyage number ::: {d['voyageDetails']['voyageNumber']}")
    # print(f"container type ::: {d['containerDetails']['type']}")
    print(f"container size ::: {d['containerDetails']['size']}")
    # print(f"score: {d['score']}")
    print(f"match:")
    for match in d['highlights']:
        match_str = " , ".join([obj['value'] for obj in match['texts'] if obj['type'] == "hit"])
        print(f"\t{match['path']} - {match_str}")
    print()

print(f"\n{len(results)} results")

CHERRY202410278290
container size ::: 40 DRY HIGH CUBE
match:
	containerDetails.size - HIGH , CUBE
	cargoDetails.commodity - HIGH
	remarks - HIGH , CUBE , High

CHERRY202411097585
container size ::: 40 HIGH CUBE CONTAINER
match:
	containerDetails.size - HIGH , CUBE
	containerDetails.type - HIGH , CUBE
	remarks - HIGH , CUBE

CHERRY202411073757
container size ::: 40 HIGH CUBE
match:
	containerDetails.size - HIGH , CUBE
	containerDetails.type - HIGH , CUBE
	remarks - HIGH , CUBE

CHERRY202410238386
container size ::: 40 HIGH CUBE CONTAINER
match:
	containerDetails.size - HIGH , CUBE
	containerDetails.type - HIGH , CUBE
	remarks - HIGH , CUBE

CHERRY202409299229
container size ::: 40 HIGH CUBE
match:
	containerDetails.size - HIGH , CUBE
	containerDetails.type - HIGH , CUBE
	remarks - High , Cube

CHERRY202411085156
container size ::: 40 HIGH CUBE
match:
	containerDetails.size - HIGH , CUBE
	containerDetails.type - HIGH , CUBE
	remarks - High , Cube

CHERRY202411047912
container size ::: 4